## Machine Learning Lab-4: Exploratory Data Analysis (100 marks)

In [94]:
# Run this cell to install the needed dependencies

!pip install pycountry

In [1]:
#This dataset contain information of hotel booking, We have performed exploratory data analysis in python to get insight 
#from the data.

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
#import seaborn as sns
import pycountry as pc
import matplotlib.ticker as mtick

pd.options.display.max_columns = None

ImportError: DLL load failed while importing _ufuncs: The specified module could not be found.

### Import and Display the first 5 rows of the dataset (5 marks)

(You can download the dataset from Canvas)

In [8]:
# Import data
data=pd.read_csv("hotel_bookings.csv")
# Enter your code here
data.head(5)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


### Data Preprocessing (10 marks)

**Copy the dataset so that our original dataset remains unchanged (1 mark)**

In [9]:
# Enter your code here
data_copy = data.copy()

**Find the size of the dataframe (1 mark)**

In [10]:
# Enter your code here
data_copy.shape

(119390, 32)

**Check if our data contains any missing  values (2 marks)**

In [11]:
## Find the missing value, show the total null values for each column and sort it in descending order
# Enter your code here
data_copy.isnull().sum().sort_values(ascending=False)[:10]

company                   112593
agent                      16340
country                      488
children                       4
reserved_room_type             0
assigned_room_type             0
booking_changes                0
deposit_type                   0
hotel                          0
previous_cancellations         0
dtype: int64

We can see that company, agent, country and children contain missing values

In [12]:
## For the missing values in company and agent, just replace them with 0

data_copy[['agent','company']] = data_copy[['agent','company']].fillna(0.0)

In [13]:
## For the missing values in the country column, replace it with mode (value that appears most often)

data_copy['country'].fillna(data_copy.country.mode().to_string(), inplace=True)

In [14]:
## For missing children value, replace it with rounded mean value

data_copy['children'].fillna(round(data_copy.children.mean()), inplace=True)

**Check if there are any rows with zero guests (No adults, children or babies)** 

In [15]:
data_copy[(data_copy.adults+data_copy.babies+data_copy.children)==0].shape

(180, 32)

In [16]:
data_copy = data_copy.drop(data_copy[(data_copy.adults+data_copy.babies+data_copy.children)==0].index)

**Find the new size of the dataframe after preprocessing (1 mark)**

In [17]:
# Enter your code here
data_copy.shape

(119210, 32)

**Checking the datatypes of each columns in the dataset (2 marks)**

In [18]:
# Enter your code here
data_copy.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                          float64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

**Convert the datatypes into integers as some are declared floats but uses only integers**

In [19]:
## convert datatype of these columns from float to integer
data_copy[['children', 'company', 'agent']] = data_copy[['children', 'company', 'agent']].astype('int64')

**Check the datatypes one more time to see if the conversion has worked or not (2 marks)**

In [20]:
# Enter your code here
data_copy.dtypes

hotel                              object
is_canceled                         int64
lead_time                           int64
arrival_date_year                   int64
arrival_date_month                 object
arrival_date_week_number            int64
arrival_date_day_of_month           int64
stays_in_weekend_nights             int64
stays_in_week_nights                int64
adults                              int64
children                            int64
babies                              int64
meal                               object
country                            object
market_segment                     object
distribution_channel               object
is_repeated_guest                   int64
previous_cancellations              int64
previous_bookings_not_canceled      int64
reserved_room_type                 object
assigned_room_type                 object
booking_changes                     int64
deposit_type                       object
agent                             

### Perform Exploratory Data Analysis (45 marks)

**Answer the following questions using the following functions. The answers for these questions are not present in the reference file. You need to do this yourself**

In [21]:
#This function gives the precentage of different values

def get_count(series, limit=None):  
    
    '''
    INPUT:
        series: Pandas Series (Single Column from DataFrame)
        limit:  If value given, limit the output value to first limit samples.
    OUTPUT:
        x = Unique values
        y = Count of unique values
    '''
    
    if limit != None:
        series = series.value_counts()[:limit]
    else:
        series = series.value_counts()
    
    x = series.index
    y = series/series.sum()*100
    
    return x.values,y.values

In [22]:
def plot(x, y, x_label=None,y_label=None, title=None, figsize=(7,5), type='bar'):
    
    '''
    INPUT:
        x:        Array containing values for x-axis
        y:        Array containing values for y-axis
        x_lable:  String value for x-axis label
        y_lable:  String value for y-axis label
        title:    String value for plot title
        figsize:  tuple value, for figure size
        type:     type of plot (default is bar plot)
        
    OUTPUT:
        Display the plot
    '''
    
    sns.set_style('darkgrid')
    
    fig, ax = plt.subplots(figsize=figsize)
    
    ax.yaxis.set_major_formatter(mtick.PercentFormatter())
    
    if x_label != None:
        ax.set_xlabel(x_label)
    
    if y_label != None:
        ax.set_ylabel(y_label)
        
    if title != None:
        ax.set_title(title)
    
    if type == 'bar':
        sns.barplot(x,y, ax = ax)
    elif type == 'line':
        sns.lineplot(x,y, ax = ax, sort=False)
        
    
    plt.show()

**Question-1 What is the booking ratio between resort hotel and city hotel? (15 marks) (5 marks for the code, 5 marks for the graph and 5 marks for the analysis)**

**Question-2 What is the percentage of booking for each year? (15 marks) (5 marks for the code, 5 marks for the graph and 5 marks for the analysis)**

**Question-3 How many bookings were cancelled? (15 marks) (5 marks for the code, 5 marks for the graph and 5 marks for the analysis)**

Hint: Pass the appropriate dataframe columns to the get_count() and plot() functions, plot the graphs and analyze the graphs to answer the questions

In [25]:
# Enter your code for question-1 here
get_count(data_copy["hotel"], limit=None)


(array(['City Hotel', 'Resort Hotel'], dtype=object),
 array([66.40634175, 33.59365825]))

In [29]:

# Enter your analysis here
plot(66.406344175, 33.59365825, x_label="City Hotel", y_label="Resort Hotel", title="Question 1", figsize=(7,5), type='bar')


NameError: name 'sns' is not defined

### booking ratio is 1/2 Resort Hotel over City Hotel. City Hotels is booked 2 times more than Resort Hotel

In [30]:
# Enter your code for question-2 here
get_count(data_copy["arrival_date_year"], limit=None)


(array([2016, 2017, 2015], dtype=int64),
 array([47.498532  , 34.07432262, 18.42714537]))

### persentage for each year 2015-18.43%  2016-47.49%   2017-37.07% 

In [34]:
# Enter your code for question-3 here
get_count(data_copy["is_canceled"], limit=None)


(array([0, 1], dtype=int64), array([62.92341247, 37.07658753]))


### 37 % of booking was canceled

### Feature Selection (20 marks)

**Remove the unwanted features (10 marks)**

Use drop() to drop the unwanted features. The unwanted features in this dataset are:

arrival_date_year, arrival_date_week_number, arrival_date_day_of_month, arrival_date_month, assigned_room_type, reserved_room_type, reservation_status_date, previous_cancellations, previous_bookings_not_canceled, reservation_status

Also pass axis=1

In [72]:
# Enter your code here
data_copy=data_copy.drop(['arrival_date_year', 'arrival_date_week_number', 'arrival_date_day_of_month', 'arrival_date_month', 
         'assigned_room_type', 'reserved_room_type', 'reservation_status_date', 'previous_cancellations', 
         'previous_bookings_not_canceled', 'reservation_status'], axis=1)
data_copy

,hotel,is_canceled,lead_time,stays_in_weekend_nights,stays_in_week_nights,adults,children,babies,meal,country,...,is_repeated_guest,booking_changes,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests
0,Resort Hotel,0,342,0,0,2,0,0,BB,PRT,...,0,3,No Deposit,0,0,0,Transient,0.00,0,0
1,Resort Hotel,0,737,0,0,2,0,0,BB,PRT,...,0,4,No Deposit,0,0,0,Transient,0.00,0,0
2,Resort Hotel,0,7,0,1,1,0,0,BB,GBR,...,0,0,No Deposit,0,0,0,Transient,75.00,0,0
3,Resort Hotel,0,13,0,1,1,0,0,BB,GBR,...,0,0,No Deposit,304,0,0,Transient,75.00,0,0
4,Resort Hotel,0,14,0,2,2,0,0,BB,GBR,...,0,0,No Deposit,240,0,0,Transient,98.00,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119385,City Hotel,0,23,2,5,2,0,0,BB,BEL,...,0,0,No Deposit,394,0,0,Transient,96.14,0,0
119386,City Hotel,0,102,2,5,3,0,0,BB,FRA,...,0,0,No Deposit,9,0,0,Transient,225.43,0,2
119387,City Hotel,0,34,2,5,2,0,0,BB,DEU,...,0,0,No Deposit,9,0,0,Transient,157.71,0,4
119388,City Hotel,0,109,2,5,2,0,0,BB,GBR,...,0,0,No Deposit,89,0,0,Transient,104.40,0,0


**Find the categorical values and remove them as well from the dataset (10 marks)**

In [73]:
# Find the categorical features
categorical_features = list(data_copy.columns[data_copy.dtypes == object])

In [74]:
# Print the categorical features here
print(categorical_features)

['hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 'deposit_type', 'customer_type']


Drop these columns as well since they contain categorical values and may hinder in training and testing the model

In [75]:
# Enter your code here
data_copy=data_copy.drop(['hotel', 'meal', 'country', 'market_segment', 'distribution_channel', 'deposit_type', 'customer_type'], axis=1)

In [76]:
# Print the dataframe one more time to check if these columns still exist in the dataframe
print(data_copy.head)

<bound method NDFrame.head of         is_canceled  lead_time  stays_in_weekend_nights  stays_in_week_nights  \
0                 0        342                        0                     0   
1                 0        737                        0                     0   
2                 0          7                        0                     1   
3                 0         13                        0                     1   
4                 0         14                        0                     2   
...             ...        ...                      ...                   ...   
119385            0         23                        2                     5   
119386            0        102                        2                     5   
119387            0         34                        2                     5   
119388            0        109                        2                     5   
119389            0        205                        2                     7  

### Create decision tree models for both regression and classification (20 marks)

**Train and test the data. Use 'is_cancelled' as the target. Use training to testing ratio as 3:1 (4 marks)**

In [92]:
# Define X and Y values for the train_test_split here

X = data_copy.drop(['is_canceled'], axis=1)
Y = data_copy['is_canceled']

In [93]:
# Use train_test_split here
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,Y, test_size=0.3, random_state=1)


ImportError: DLL load failed while importing _ufuncs: The specified module could not be found.

**Use decision tree model (regression) to test the data (4 marks)**

In [ ]:


# Enter your code here



**Predict the value of the 45th record in x_train (4 marks)**

In [ ]:


# Enter your code here



**Use decision tree model (classification) to test the data (4 marks)**

In [ ]:


# Enter your code here



**Predict the value of the 75th record in x_train (4 marks)**

In [ ]:


# Enter your code here

